# Data preprocessing notebook for spatio-temporal density modelling

Author: Peter Mørch Groth

This notebook performs preprocessing of the INSERT_NAME dataset.

This is comprised of several steps:
- The data is cleaned and columns of interest are extracted (latitude, longitude, and timestamp at beginning of trips).
- The data is binned into 2 hour intervals. 
- In total, three tensors are created: 
    - $X$, which contain the latitude/longitude pairs for each interval.
    - $U$, which are the gridded observation the time intervals. The grid is 20 by 20.
    - $N$, which is a mask denoting the number of observations for each time interval in $X$. This is to deal with the different number at each interval.
- The data is normalised.
- The tensors are saved.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import torch

### Load and clean data

In [2]:
# Load file
filename = 'LTMZones1.csv'
f = lambda x: (x.replace(",",".")) 
rawdata = pd.read_csv(filename, sep=';', decimal=',')#converters = {'Latitude (Start)':f, 'Longitude (Start)':f})

/home/peterg/miniconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (3,4,5,6,12,13) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
# Show data
display(rawdata)

,TurID,BilID,PersonID,Latitude (Start),Longitude (Start),Latitude (Slut),Longitude (Slut),Reservationstidspunkt,Start tidspunkt,Slut tidspunkt,Alder,Køn,Batteristatus (start),Batteristatus (slut),Km kørt,FromZoneID,ToZoneID
0,1,WBY1Z21040V307905,32614,55.6703,12.5861,55.653,12.5447,16/08/2017 14:40,16/08/2017 14:34,16/08/2017 14:52,32,M,94,91,5,103142,102851
1,2,WBY1Z21040V308214,44363,55.687,12.586,55.7267,12.5539,16/08/2017 14:42,16/08/2017 14:38,16/08/2017 15:05,25,M,48,42,7,102212,157231
2,3,WBY1Z21070V307929,38610,55.6924,12.5831,55.7757,12.514,31/07/2017 07:28,31/07/2017 07:21,01/08/2017 12:49,21,M,0,0,19,102213,173051
3,4,WBY1Z21040V307869,37425,55.6303,12.6489,55.7069,12.5871,31/07/2017 20:31,31/07/2017 20:31,01/08/2017 09:13,30,M,0,0,18,185203,102341
4,5,WBY1Z21030V307877,24168,55.6936,12.5382,55.6775,12.4892,31/07/2017 20:48,31/07/2017 20:44,01/08/2017 08:52,28,M,0,0,5,102454,102771
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117830,120481,WBY1Z21010V307831,45879,"55,645887","12,605008","55,6425","12,60724",09/11/2017 15:47,09/11/2017 15:41,09/11/2017 16:00,26,M,47,39,5,103252,103251
117831,120482,WBY1Z21080V308247,47793,"55,66293","12,5143","55,665967","12,494803",09/11/2017 15:47,09/11/2017 15:45,09/11/2017 16:19,20,M,89,79,6,102724,102732
117832,120483,WBY1Z21070V308045,28924,"55,630252","12,648886","55,699598","12,530801",09/11/2017 15:25,09/11/2017 15:24,09/11/2017 16:21,43,M,43,13,27,185203,102541
117833,120484,WBY1Z21040V307855,49796,"55,68031","12,552588","55,672008","12,495394",09/11/2017 15:25,09/11/2017 15:24,09/11/2017 15:42,19,M,85,76,5,147112,102771


In [4]:
# Extract pickup location (lat/long) and pickup time
X = rawdata[['Start tidspunkt', 'Latitude (Start)', 'Longitude (Start)']]
X = X.rename(columns={'Latitude (Start)':'lat', 'Longitude (Start)':'lon', 'Start tidspunkt':'t'})
print(f'Total number of unfiltered datapoints: {len(X)}')

# Find and remove missing values (denoted by '-')
rm_idx = rawdata[(rawdata['Longitude (Start)'] == '-') | (rawdata['Longitude (Slut)'] == '-')].index
X = X.drop(index=rm_idx)

# Replace commas with decimal points where read_csv failed
def fix(x):
    if type(x) is str:
        x = float(x.replace(',', '.'))
    return x
X.lon = X.lon.apply(lambda x: fix(x))
X.lat = X.lat.apply(lambda x: fix(x))
print(f'Total number of filtered datapoints: {len(X)}')

assert X.lon.dtype == 'float64'
assert X.lat.dtype == 'float64'

# Format columns, sort, reindex
X.t = pd.to_datetime(X.t)
X = X.sort_values(by='t')
# Extract interval of interest. 13/10/2017-31/10/2017
X = X[(X['t'] > pd.Timestamp(year=2017, month=10, day=13, hour=0, minute=0, second=0)) & (X['t'] < pd.Timestamp(year=2017, month=11, day=1, hour=0, minute=0, second=1))]

X = X.set_index('t')
# Determine grid boundaries
full_data = False
if full_data:
    latMax = X.lat.max() # 56.03
    latMin = X.lat.min() # 55.53
    lonMax = X.lon.max() # 12.68
    lonMin = X.lon.min() # 12.07
else:
    latMax = 55.73
    latMin = 55.62
    lonMax = 12.67
    lonMin = 12.52
    # Remove datapoints outside the boundary
    rm_idx = X[(X.lat > latMax) | (X.lat < latMin) | (X.lon > lonMax) | (X.lon < lonMin)].index
    X = X.drop(index=rm_idx)    
print(f'Total datapoints to be used for modeling: {len(X)}')

# Specify grid size
k = 20

Total number of unfiltered datapoints: 117835
Total number of filtered datapoints: 117833
Total datapoints to be used for modeling: 14781


### Batch data into user-defined hour intervals

In [5]:
# Batch datapoints into two-hour intervals
freq = '120min'
start = X.resample(freq).count().index[0]
end   = X.resample(freq).count().index[-1]
N  = X.resample(freq).count().values[:,0]
idx_keep = np.where(N != 0)[0]
Nmax = max(N)
T    = len(N)

### Create tensors

In [6]:
# Create X Tensor:
# Format [N, T, Nt, D]
# N: number of sequences (1)
# T: time index, two hour intervals
# N_t: coordinates at time t (lat, lon)
# D: dimension of data (2)

# Group in 2h intervals
grouped = X.resample(freq)
# Allocate tensor
Xt = torch.zeros((1, T, Nmax, 2))
# Fill tensor
for i, (name, group) in enumerate(grouped):
    values = torch.from_numpy(group.values)
    Xt[0, i, :N[i], 0] = values[:,1]
    Xt[0, i, :N[i], 1] = values[:,0]

In [7]:
# Create U Tensor:

# Allocate
U = torch.zeros((1, T, k, k))
for i in range(T-1):
    # Extract current lat/lon
    temp = Xt[:, i, :N[i], :] 
    x = temp.numpy()[:,:,0][0]
    y = temp.numpy()[:,:,1][0]
    # Compute 2d histogram of current values in given grid
    u, r1, r2 = np.histogram2d(x=x, y=y, bins=[k,k], range=[[lonMin, lonMax], [latMin, latMax]])
    # Normalise such that values sum to 1
    if u.sum() > 0:
        u /= u.sum()
    # Fill
    U[:, i+1, :, :] =  torch.from_numpy(np.rot90(u).copy())

In [8]:
# First U (zeros) is discarded alongside first X
Xt = Xt[:, 1:, :, :]
U  = U[:, 1:, :, :]
if 0 in idx_keep:
    idx_keep = idx_keep[1:]
idx_keep -= 1
N = N[1:]
N = N[idx_keep]

# Remove timestamps with no data points
Xt = Xt[:, idx_keep, :, :]
U  = U[:, idx_keep, :, :]

print(f'Shape of X matrix: {Xt.shape}')
print(f'Shape of U matrix: {U.shape}')

Shape of X matrix: torch.Size([1, 226, 167, 2])
Shape of U matrix: torch.Size([1, 226, 20, 20])


### Normalise data

In [ ]:
# Compute quantities and normalise data
lonmean = X.lon.mean()
latmean = X.lat.mean()
lonstd  = X.lon.std()
latstd  = X.lat.std()
for i in range(Xt.shape[1]):
    Xt[0, i, :N[i], 0] = (Xt[0, i, :N[i], 0] - torch.ones(Xt[0, i, :N[i], 0].shape)*lonmean)/lonstd
    Xt[0, i, :N[i], 1] = (Xt[0, i, :N[i], 1] - torch.ones(Xt[0, i, :N[i], 1].shape)*latmean)/latstd
# Show values
print(f'Latitude mean: \t{latmean}')
print(f'Latitude std: \t{latstd}')
print(f'Longitude mean: {lonmean}')
print(f'Longitude std: \t{lonstd}')

### Split into training, test and validation sets

In [10]:
lb = Xt.shape[1]//2
ub = lb + Xt.shape[1]//4
X_train_tensor = Xt[:, :lb,   :, :]
X_test_tensor  = Xt[:, lb:ub, :, :]
X_val_tensor   = Xt[:, ub:,   :, :]
U_train_tensor = U[:, :lb,    :, :]
U_test_tensor  = U[:, lb:ub,  :, :]
U_val_tensor   = U[:, ub:     :, :]
N_t_train      = N[:lb]
N_t_test       = N[lb:ub]
N_t_val        = N[ub:]

In [38]:
print(f'Latitude mean: \t{latmean}')
print(f'Latitude std: \t{latstd}')
print(f'Longitude mean: {lonmean}')
print(f'Longitude std: \t{lonstd}')

Latitude mean: 	55.67777941810432
Latitude std: 	0.022970362839316014
Longitude mean: 12.572154878154388
Longitude std: 	0.03018063809393041


### Save Tensors

In [49]:
torch.save(X_train_tensor, './data/X_train_tensor.pt')
torch.save(X_test_tensor, './data/X_test_tensor.pt')
torch.save(X_val_tensor, './data/X_val_tensor.pt')
torch.save(U_train_tensor, './data/U_train_tensor.pt')
torch.save(U_test_tensor, './data/U_test_tensor.pt')
torch.save(U_val_tensor, './data/U_val_tensor.pt')
np.save('./data/N_t_train.npy', N_t_train)
np.save('./data/N_t_test.npy', N_t_test)
np.save('./data/N_t_val.npy', N_t_val)